In [ ]:
import plotly.graph_objects as go
import pandas as pd
import rasterio
import geopandas as gpd
from shapely.geometry import Point
import plotly.express as px

In [ ]:
biome_data = gpd.read_file("assets\Biomas_250mil\lm_bioma_250.shp")

In [ ]:
OSD_df = pd.read_parquet("generated_files/bio_variables_dataframe_occurence.parquet")

# Creating Geometry
OSD_df['geometry'] = list(zip(OSD_df["Longitude"], OSD_df["Latitude"]))
OSD_df['geometry'] = OSD_df["geometry"].apply(Point)

# Create the geodataframe
OSD_geoframe = gpd.GeoDataFrame(
    OSD_df,
    crs = {'init': 'epsg:4326'},
    geometry = OSD_df['geometry']
)
OSD_geoframe = OSD_geoframe.to_crs("EPSG:4326")
OSD_geoframe.reset_index(drop=True, inplace = True)

OSD_geoframe['Year'] = OSD_geoframe['Data'].dt.year
OSD_df = pd.DataFrame(OSD_geoframe)

In [ ]:
img_config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'species_distribution_with_biomes',
    'scale':4 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

In [ ]:
biome_data

In [ ]:
color_discrete_map={
        'Amazônia':"#3ec752",
        "Caatinga":"#a19443",
        "Cerrado":"#e09b5e",
        "Mata Atlântica":"#33d6b6",
        "Pampa":"#b0e6a8",
        "Pantanal":"#9e1f1c"
}

In [ ]:
fig = px.choropleth_mapbox(
        biome_data,
        locations=biome_data.index,
        center={"lat": -16.95, "lon": -47.78},
        geojson=biome_data.geometry.__geo_interface__,
        color="Bioma",
        opacity=0.6,
        zoom=3,
        hover_data={"Bioma": True},
        mapbox_style="carto-positron"
)

trace = px.scatter_mapbox(
    OSD_df,
    lat="Latitude",
    lon="Longitude",
    color_discrete_sequence=[px.colors.qualitative.Plotly[1]],
    zoom=4,
    height=500,
    opacity=0.9
).data[0]
fig.add_trace(trace)

fig.update_layout(autosize=True,margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_geos(
    visible=False,
    resolution=50,
    lataxis_gridcolor="red",
    lataxis_showgrid=True,
    lataxis_dtick=15,
    lonaxis_showgrid=True,
    lonaxis_dtick=15,
)
#fig.write_image('figure.png', scale=2)
fig.show(config=img_config)
fig.show()